### Web Scrapping

In [ ]:
# !pip install --upgrade pip
# !pip install beautifulsoup4 --upgrade
# pip install lxml
# pip install html5lib

In [9]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import re
import requests
from urllib.request import Request, urlopen
from urllib import parse
import glob

#### External HTML File From Browser Element Section 

In [44]:
lang = "indonesian"  # indonesian, turkish, russian, english, urdu, hindi, bangla
surah = "al jumu'a"  # al faatiha, al baqara, aal i imraan ...
folder = f"{surah}-62"  # al faatiha-1, al baqara-2, aal i imraan-3 ...
file = f"{surah.lower()}_{lang.lower()}"

In [45]:
HTMLFile = open(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Data/{folder.lower().capitalize()}/{file.lower()}.html", "r") # ayahs-block

content = HTMLFile.read()

soup = BeautifulSoup(content, 'lxml')

ayah_word = soup.findAll('span', attrs={"class":"single-word"})

word_list = []
for word in ayah_word:
    var1 = word.find('span', attrs={"class":"word-arabic word-arabic-uthmani-hafs"})
    #try:
    #    #arabic = var1.get_text()
    #    arabic = var1.string
    #except:
    #    arabic = ""
    #arabic = var1.get_text()
    arabic = var1.string
    var2 = word.find('span', attrs={"class":"word-transliteration"})
    transliterate = var2.string
    var3 = word.find('span', attrs={"class":"word-translation"})
    translate = var3.string
    word_list.append((arabic, transliterate, translate))

df_word_list = pd.DataFrame(word_list)
df_word_list.rename(columns={0:"arabic", 1:"transliterate", 2:f"translate_{lang.lower()}"}, inplace=True)
df_word_list

,arabic,transliterate,translate_indonesian
0,يُسَبِّحُ,yusabbihu,bertasbih
1,لِلَّهِ,lillahi,kepada Allah
2,مَا,ma,apa
3,فِي,fi,pada
4,ٱلسَّمَٰوَٰتِ,al-samawati,langit(jamak)
...,...,...,...
170,وَمِنَ,wamina,dan dari
171,ٱلتِّجَٰرَةِۚ,al-tijarati,perniagaan
172,وَٱللَّهُ,wal-lahu,dan Allah
173,خَيۡرُ,khayru,sebaik-baik


In [46]:
df_word_list.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_{lang.lower().capitalize()}.xlsx", index=False)

#### Merge Word All Translate

In [47]:
all_trans_file = glob.glob(f"{surah.lower().capitalize()}*")
all_trans_file

["Al jumu'a_Arabic_Transliterate_Translate_Words_Bangla.xlsx",
 "Al jumu'a_Arabic_Transliterate_Translate_Words_Hindi.xlsx",
 "Al jumu'a_Arabic_Transliterate_Translate_Words_Urdu.xlsx",
 "Al jumu'a_Arabic_Transliterate_Translate_Words_English.xlsx",
 "Al jumu'a_Arabic_Transliterate_Translate_Words_Russian.xlsx",
 "Al jumu'a_Arabic_Transliterate_Translate_Words_Turkish.xlsx",
 "Al jumu'a_Arabic_Transliterate_Translate_Words_Indonesian.xlsx"]

In [48]:
df_master = pd.read_excel(all_trans_file[0])
for i in all_trans_file[1:]:
    df_var = pd.read_excel(f"{i}")
    #df_master = pd.merge(df_master, df_var, how="left", on=["arabic","transliterate"])
    df_master = pd.concat([df_master,df_var.iloc[:,2]], axis=1)
    #df_master = df_master.join(df_var, how="left", lsuffix=["arabic","transliterate"],rsuffix=["arabic","transliterate"])
    #df_master.drop_duplicates(inplace=True)
    df_master.reset_index(drop=True, inplace=True)
df_master = df_master[["arabic","transliterate","translate_english","translate_urdu","translate_hindi","translate_indonesian","translate_bangla","translate_turkish","translate_russian"]]
df_master

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,يُسَبِّحُ,yusabbihu,Glorifies,تسبیح کررہی ہے,तस्बीह करती है,bertasbih,মহিমা ঘোষণা করে,tesbih etmektedir,Восславляют
1,لِلَّهِ,lillahi,Allah,اللہ ہی کے لیے,अल्लाह के लिए,kepada Allah,আল্লাহরই,Allah'ı,Аллаха
2,مَا,ma,whatever,ہر وہ چیز جو,हर वो चीज़ जो,apa,যা,ne varsa,"то, что"
3,فِي,fi,(is) in,میں,*,pada,মধ্যে (আছে),göklerde,в
4,ٱلسَّمَٰوَٰتِ,al-samawati,the heavens,آسمان میں ہے,आसमानों में है,langit(jamak),আকাশ জগতের,*,"небесах,"
...,...,...,...,...,...,...,...,...,...
170,وَمِنَ,wamina,and from,سے اور,*,dan dari,ও থেকে,ve,и
171,ٱلتِّجَٰرَةِۚ,al-tijarati,(any) transaction,تجارت (سے),और तिजारत से,perniagaan,ব্যবসা,ticaretten,торговля».
172,وَٱللَّهُ,wal-lahu,And Allah,اور اللہ,और अल्लाह,dan Allah,এবং আল্লাহই,ve Allah,И Аллах –
173,خَيۡرُ,khayru,(is the) Best,بہترین,बेहतर है,sebaik-baik,উত্তম,en hayırlısıdır,лучший


In [49]:
df_master.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx", index=False)

#### Convert Uthmani To Plain Buckwalter

In [32]:
import pandas as pd
import re
import camel_tools.tokenizers.word
from camel_tools.utils.charmap import CharMapper
from lang_trans.arabic import buckwalter

In [33]:
surah = "al jumu'a"  # al faatiha, al baqara, aal i imraan ...
folder = f"{surah}-62"  # al faatiha-1, al baqara-2, aal i imraan-3 ...

In [34]:
df_surah = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/{folder.lower().capitalize()}/{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_surah

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,يُسَبِّحُ,yusabbihu,Glorifies,تسبیح کررہی ہے,तस्बीह करती है,bertasbih,মহিমা ঘোষণা করে,tesbih etmektedir,Восславляют
1,لِلَّهِ,lillahi,Allah,اللہ ہی کے لیے,अल्लाह के लिए,kepada Allah,আল্লাহরই,Allah'ı,Аллаха
2,مَا,ma,whatever,ہر وہ چیز جو,हर वो चीज़ जो,apa,যা,ne varsa,"то, что"
3,فِي,fi,(is) in,میں,*,pada,মধ্যে (আছে),göklerde,в
4,ٱلسَّمَٰوَٰتِ,al-samawati,the heavens,آسمان میں ہے,आसमानों में है,langit(jamak),আকাশ জগতের,*,"небесах,"
...,...,...,...,...,...,...,...,...,...
170,وَمِنَ,wamina,and from,سے اور,*,dan dari,ও থেকে,ve,и
171,ٱلتِّجَٰرَةِۚ,al-tijarati,(any) transaction,تجارت (سے),और तिजारत से,perniagaan,ব্যবসা,ticaretten,торговля».
172,وَٱللَّهُ,wal-lahu,And Allah,اور اللہ,और अल्लाह,dan Allah,এবং আল্লাহই,ve Allah,И Аллах –
173,خَيۡرُ,khayru,(is the) Best,بہترین,बेहतर है,sebaik-baik,উত্তম,en hayırlısıdır,лучший


In [35]:
# Arabic Encoding Extended
# -*- coding: utf-8 -*-

# Arabic Transliteration based on Buckwalter
# dictionary source is buckwalter2unicode.py http://www.redhat.com/archives/fedora-extras-commits/2007-June/msg03617.html 

buck2uni = {"'": u"\u0621", # hamza-on-the-line
            "|": u"\u0622", # madda
            ">": u"\u0623", # hamza-on-'alif
            "&": u"\u0624", # hamza-on-waaw
            "<": u"\u0625", # hamza-under-'alif
            "}": u"\u0626", # hamza-on-yaa'
            "A": u"\u0627", # bare 'alif
            "b": u"\u0628", # baa'
            "p": u"\u0629", # taa' marbuuTa
            "t": u"\u062A", # taa'
            "v": u"\u062B", # thaa'
            "j": u"\u062C", # jiim
            "H": u"\u062D", # Haa'
            "x": u"\u062E", # khaa'
            "d": u"\u062F", # daal
            "*": u"\u0630", # dhaal
            "r": u"\u0631", # raa'
            "z": u"\u0632", # zaay
            "s": u"\u0633", # siin
            "$": u"\u0634", # shiin
            "S": u"\u0635", # Saad
            "D": u"\u0636", # Daad
            "T": u"\u0637", # Taa'
            "Z": u"\u0638", # Zaa' (DHaa')
            "E": u"\u0639", # cayn
            "g": u"\u063A", # ghayn
            "_": u"\u0640", # taTwiil
            "f": u"\u0641", # faa'
            "q": u"\u0642", # qaaf
            "k": u"\u0643", # kaaf
            "l": u"\u0644", # laam
            "m": u"\u0645", # miim
            "n": u"\u0646", # nuun
            "h": u"\u0647", # haa'
            "w": u"\u0648", # waaw
            "Y": u"\u0649", # 'alif maqSuura
            "y": u"\u064A", # yaa'
            "F": u"\u064B", # fatHatayn
            "N": u"\u064C", # Dammatayn
            "K": u"\u064D", # kasratayn
            "a": u"\u064E", # fatHa
            "u": u"\u064F", # Damma
            "i": u"\u0650", # kasra
            "~": u"\u0651", # shaddah
            "o": u"\u0652", # sukuun
            "^": u"\u0653", # maddah
            "#": u"\u0654", # hamzaabove            
            "`": u"\u0670", # dagger 'alif
            "{": u"\u0671", # waSla
            "P": u"\u067E", # arabicletterpeh
            "J": u"\u0686", 
            "V": u"\u06A4", # arabicletterveh
            "G": u"\u06AF", # arabiclettergaf
            ":": u"\u06DC", # smallhighseen
            "@": u"\u06DF", # smallhighroundedzero
            "\"": u"\u06E0", # smallhighuprightrectangularzero
            "[": u"\u06E2", # smallhighmeemisolatedform
            ";": u"\u06E3", # smalllowseen
            ",": u"\u06E5", # smallwaw
            ".": u"\u06E6", # smallya
            "!": u"\u06E8", # smallhighnoon
            "-": u"\u06EA", # emptycentrelowstop
            "+": u"\u06EB", # emptycentrehighstop
            "%": u"\u06EC", # roundedhighstopwithfilledcentre
            "]": u"\u06ED", # smalllowmeem           
}

def transString(string, reverse=0):
    '''Given a Unicode string, transliterate into Buckwalter. To go from
    Buckwalter back to Unicode, set reverse=1'''

    for k, v in buck2uni.items():
      if not reverse:
            string = string.replace(v, k)
      else:
            string = string.replace(k, v)

    return string

In [36]:
def convert_uthmani_to_plain(word):
    buckwalter_letter_select = re.compile(r"[AbptvjHxdrzsSDTZEgfqklmnhwYyFNKaui~o'|>&<}\*\$`]+", re.UNICODE)
    buckwalter_word = transString(word)
    buckwalter_plain = re.findall(buckwalter_letter_select, buckwalter_word)
    word = "".join(buckwalter_plain)

    return word

In [37]:
buckwalter = df_surah.arabic.apply(lambda x: convert_uthmani_to_plain(x))
df_surah.insert(1, "buckwalter_word_tanzil_plain", buckwalter)
df_surah

,arabic,buckwalter_word_tanzil_plain,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,يُسَبِّحُ,yusab~iHu,yusabbihu,Glorifies,تسبیح کررہی ہے,तस्बीह करती है,bertasbih,মহিমা ঘোষণা করে,tesbih etmektedir,Восславляют
1,لِلَّهِ,lil~ahi,lillahi,Allah,اللہ ہی کے لیے,अल्लाह के लिए,kepada Allah,আল্লাহরই,Allah'ı,Аллаха
2,مَا,maA,ma,whatever,ہر وہ چیز جو,हर वो चीज़ जो,apa,যা,ne varsa,"то, что"
3,فِي,fiy,fi,(is) in,میں,*,pada,মধ্যে (আছে),göklerde,в
4,ٱلسَّمَٰوَٰتِ,ls~ama`wa`ti,al-samawati,the heavens,آسمان میں ہے,आसमानों में है,langit(jamak),আকাশ জগতের,*,"небесах,"
...,...,...,...,...,...,...,...,...,...,...
170,وَمِنَ,wamina,wamina,and from,سے اور,*,dan dari,ও থেকে,ve,и
171,ٱلتِّجَٰرَةِۚ,lt~ija`rapi,al-tijarati,(any) transaction,تجارت (سے),और तिजारत से,perniagaan,ব্যবসা,ticaretten,торговля».
172,وَٱللَّهُ,wall~ahu,wal-lahu,And Allah,اور اللہ,और अल्लाह,dan Allah,এবং আল্লাহই,ve Allah,И Аллах –
173,خَيۡرُ,xayru,khayru,(is the) Best,بہترین,बेहतर है,sebaik-baik,উত্তম,en hayırlısıdır,лучший


In [ ]:
#df_surah.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Buckwalter_Plain_Word_Languages_Concat.xlsx", index=False)

#### Merge Surah All Translate

In [ ]:
def add_info(df,surah_name, surah_num, index):
    for i in range(len(df)):
        df["surah latin"] = surah_name
        df["surah"] = surah_num
        df.loc[i,"index"] = i+index
    return df

In [ ]:
df_faatir = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Faatir-35/Faatir_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_faatir

In [ ]:
df_faatir = add_info(df_faatir, "Fatir", 35, 56320)
df_faatir = df_faatir.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_faatir

In [ ]:
df_yaseen = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Yaseen-36/Yaseen_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_yaseen

In [ ]:
df_yaseen = add_info(df_yaseen, "Ya Seen", 36, 57102)
df_yaseen = df_yaseen.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_yaseen

In [ ]:
df_saaffaat = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/As Saaffaat-37/As saaffaat_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_saaffaat

In [ ]:
df_saaffaat = add_info(df_saaffaat, "As-Saffat", 37, 57836)
df_saaffaat = df_saaffaat.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_saaffaat

In [ ]:
df_saad = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Saad-38/Saad_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_saad

In [ ]:
df_saad = add_info(df_saad, "Sad", 38, 58705)
df_saad = df_saad.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_saad

In [ ]:
df_zumar = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Az Zumar-39/Az zumar_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_zumar

In [ ]:
df_zumar = add_info(df_zumar, "Az-Zumar", 39, 59444)
df_zumar = df_zumar.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_zumar

In [ ]:
df_surah_merge = pd.concat([df_faatir,df_yaseen,df_saaffaat,df_saad,df_zumar], axis=0)
df_surah_merge

In [ ]:
df_surah_merge.to_excel("Faatir_Yaseen_Saaffaat_Saad_Zumar_Surah_Translate_Concat.xlsx", index=False)